In [1]:
import pandas as pd

dfStatsEuwBefore = pd.read_csv("statsEUWbefore.csv")
dfStatsEuwAfter = pd.read_csv("statsEUWafter.csv")
dfStatsNaBefore = pd.read_csv("statsNAbefore.csv")
dfStatsNaAfter = pd.read_csv("statsNAafter.csv")

In [2]:
def getServerStats(before,after):
    
    #Formating numbers
    before['nbGames'] = before.apply(lambda x: "{:,}".format(x['nbGames']), axis=1)
    before['nbGameWithLead'] = before.apply(lambda x: "{:,}".format(x['nbGameWithLead']), axis=1)
    
    after['nbGames'] = after.apply(lambda x: "{:,}".format(x['nbGames']), axis=1)
    after['nbGameWithLead'] = after.apply(lambda x: "{:,}".format(x['nbGameWithLead']), axis=1)
    
    #adding a "%" to every percentage values
    before.percentageLeadingTeam = before.percentageLeadingTeam.apply(lambda s: str(s)+"%")
    before.percentageTotalGame = before.percentageTotalGame.apply(lambda s: str(s)+"%")
    before.redTeamWinrate = before.redTeamWinrate.apply(lambda s: str(s)+"%")
    before.winrateLeadingTeam = before.winrateLeadingTeam.apply(lambda s: str(s)+"%")
    
    after.percentageLeadingTeam = after.percentageLeadingTeam.apply(lambda s: str(s)+"%")
    after.percentageTotalGame = after.percentageTotalGame.apply(lambda s: str(s)+"%")
    after.redTeamWinrate = after.redTeamWinrate.apply(lambda s: str(s)+"%")
    after.winrateLeadingTeam = after.winrateLeadingTeam.apply(lambda s: str(s)+"%")
    
    #Indexing by meta
    before = before.set_index(before["Meta"]).drop(columns=['Meta'])
    after = after.set_index(after["Meta"]).drop(columns=['Meta'])
    
    del(before.index.name)
    del(after.index.name)
    
    #Merging dataframes and transposing it
    df = pd.concat([before,after], axis=1, keys=['Before 8.15','Since 8.15']).T
    
    #Swap index
    df = df.set_index(df.index.swaplevel(0,1)).sort_index()
    
    #Dirty reordering
    df["order"] = 0
    rightOrder = ['nbGames', 'percentageTotalGame', 'nbGameWithLead', 'percentageLeadingTeam', 'winrateLeadingTeam' ,'redTeamWinrate']
    a = 0
    for i in rightOrder:
        a += 1
        df["order"][i]["Before 8.15"] = a*2
        df["order"][i]["Since 8.15"] = a*2+1
    df = df.sort_values(["order"]).drop("order",axis=1)
    
    #Rename index
    df.rename({"nbGames": "Games", "percentageTotalGame":"Percentage games","nbGameWithLead":"Games with lead","percentageLeadingTeam":"Percentage of games with lead","winrateLeadingTeam":"Winrate for leading team","redTeamWinrate":"Red team winrate"}, axis='index')
    
    return df

In [3]:
euStats = getServerStats(dfStatsEuwBefore,dfStatsEuwAfter)
naStats = getServerStats(dfStatsNaBefore,dfStatsNaAfter)

data = pd.concat([euStats,naStats],axis=1, keys=['EUW', 'NA'])

def Color_server(x):
    r = 'background-color: rgba(255, 0, 0, 0.1)'
    b = 'background-color: rgba(0, 0, 255, 0.1)'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1["EUW"] = r
    df1["NA"] = b
    return df1    
data.style.apply(Color_server, axis=None)